<a href="https://colab.research.google.com/github/Blacksmith1111/Optics/blob/main/HomeTask1v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

class MyModulator:
  def __init__(self, bits_number, mod_type = "QPSK", snr = 10):
    self.mod_type = mod_type
    #self.bits_stream = np.random.binomial(n = 1, p = 0.5, size = bits_number)
    self.bits_stream = [1,1,1,0,1,0,1,0,0,0,0,0]
    self.snr = snr

  def bits_to_symbols(self):
    self.symbols = []
    if self.mod_type == "16QAM":
      self.symbols = np.array([self.bits_stream[i : i + 4] if i + 4 <= len(self.bits_stream) else 0 for i in range(0, len(self.bits_stream), 4)])
      self.symbols = self.symbols if len(self.bits_stream) % 4 == 0 else self.symbols[0:-1]
      #if len(self.bits_stream) % 4 != 0:
      #for i in range(0, len(self.bits_stream), 4):
        #if i + 4 <= len(self.bits_stream):
            #self.symbols.append(self.bits_stream[i:i+4])
    elif self.mod_type == "QPSK":
      self.symbols = np.array([self.bits_stream[i : i + 2] if i + 2 <= len(self.bits_stream) else 0 for i in range(0, len(self.bits_stream), 2)])
      self.symbols = self.symbols if len(self.bits_stream) % 2 == 0 else self.symbols[0:-1]
      #for i in range(0, len(self.bits_stream), 2):
       # if i + 2 <= len(self.bits_stream):
        #  self.symbols.append(self.bits_stream[i:i+2])
    #self.symbols = np.array(self.symbols)
    elif self.mod_type == "64QAM":
      self.symbols = np.array([self.bits_stream[i : i + 6] if i + 6 <= len(self.bits_stream) else 0 for i in range(0, len(self.bits_stream), 6)])
      self.symbols = self.symbols if len(self.bits_stream) % 6 == 0 else self.symbols[0:-1]
    return self.symbols

  def mapper(self):
    self.I = np.zeros(len(self.symbols))
    self.Q = np.zeros(len(self.symbols))
    self.bin_vals = np.zeros(self.symbols.shape)
    if self.mod_type == "16QAM":
      Levels = np.array([1, 3])
      for index, symbol in enumerate(self.symbols):
        self.bin_vals[index] = symbol
        self.I[index] = (Levels[0] if (str(symbol[1]) + str(symbol[3]) == "11" or  str(symbol[1]) + str(symbol[3]) == "10") else Levels[1]) * (-1 if str(symbol[0]) + str(symbol[2]) == "01" or str(symbol[0]) + str(symbol[2]) == "00" else 1)
        self.Q[index] = (Levels[0] if str(symbol[1]) + str(symbol[3]) == "11" or str(symbol[1]) + str(symbol[3]) == "01" else Levels[1]) * (-1 if str(symbol[0]) + str(symbol[2]) == "00" or str(symbol[0]) + str(symbol[2]) == "10" else 1)

    elif self.mod_type == "QPSK":
      Levels = np.array([-1, 1])
      for index, symbol in enumerate(self.symbols):
        self.bin_vals[index] = symbol
        self.I[index] = Levels[0] if symbol[0] == 0 else Levels[1]
        self.Q[index] = Levels[0] if symbol[1] == 0 else Levels[1]

    elif self.mod_type == "64QAM":
      dict_64 = {"000000": -7 - 7j, "000001": -7 - 5j, "000010":-7 - 1j, "000011":-7 - 3j,
                          "000100":-7 + 7j, "000101":-7 + 5j, "000110":-7 + 1j, "000111":-7 + 3j,
                          "001000":-5 - 7j, "001001":-5 - 5j, "001010":-5 - 1j, "001011":-5 - 3j,
                          "001100":-5 + 7j, "001101":-5 + 5j, "001110":-5 + 1j, "001111":-5 + 3j,
                          "010000":-1 - 7j, "010001":-1 - 5j, "010010":-1 - 1j, "010011":-1 - 3j,
                          "010100":-1 + 7j, "010101":-1 + 5j, "010110":-1 + 1j, "010111":-1 + 3j,
                          "011000":-3 - 7j, "011001":-3 - 5j, "011010":-3 - 1j, "011011":-3 - 3j,
                          "011100":-3 + 7j, "011101":-3 + 5j, "011110":-3 + 1j, "011111":-3 + 3j,
                          "100000":7 - 7j, "100001":7 - 5j, "100010":7 - 1j, "100011":7 - 3j,
                          "100100":7 + 7j, "100101":7 + 5j, "100110":7 + 1j, "100111":7 + 3j,
                          "101000":5 - 7j, "101001":5 - 5j, "101010":5 - 1j, "101011":5 - 3j,
                          "101100":5 + 7j, "101101":5 + 5j, "101110":5 + 1j, "101111":5 + 3j,
                          "110000":1 - 7j, "110001":1 - 5j, "110010":1 - 1j, "110011":1 - 3j,
                          "110100":1 + 7j, "110101":1 + 5j, "110110":1 + 1j, "110111":1 + 3j,
                          "111000":3 - 7j, "111001":3 - 5j, "111010":3 - 1j, "111011":3 - 3j,
                          "111100":3 + 7j, "111101":3 + 5j, "111110":3 + 1j, "111111":3 + 3j}
      for index,symbol in enumerate(self.symbols):
        self.I[index] = np.real(dict_64[str(symbol).replace(" ", "")[1:-1]])
        self.Q[index] = np.imag(dict_64[str(symbol).replace(" ", "")[1:-1]])

    self.mapped = self.I + 1j*self.Q
    return self.mapped

  def plot_constellation(self):
    plt.figure(figsize=(8, 8))
    plt.scatter(np.real(self.mapped), np.imag(self.mapped), color='red')
    for (x, y, txt) in zip(np.real(self.mapped), np.imag(self.mapped), self.bin_vals):
      plt.text(x, y, str(txt), fontsize=13)

    if self.mod_type == "16QAM":
      plt.title('16QAM Constellation')
      plt.xlim(-4, 4)
      plt.ylim(-4, 4)
    elif self.mod_type == "QPSK":
      plt.title('QPSK Constellation')
      plt.xlim(-4, 4)
      plt.ylim(-4, 4)

    plt.xlabel('I')
    plt.ylabel('Q')
    plt.grid(True)
    plt.axhline(0, color='black', lw=0.5)
    plt.axvline(0, color='black', lw=0.5)
    plt.show()





In [41]:
class MyDemodulator:
  def __init__(self, mapped_symbols, mod_type = "QPSK"):
    self.mod_type = mod_type
    self.mapped = mapped_symbols
    if self.mod_type == "16QAM":
      self.symbol = np.zeros(4)
    elif self.mod_type == "QPSK":
      self.symbol = np.zeros(2)

  def demodulate(self):
    out_symbol = []
    self.out_bits = []
    ref_arr = []
    if self.mod_type == "16QAM":
      dict_16 = {-3 + 3j: "1000", -3 + 1j: "0011", -3 - 3j: "0000", -3 - 1j: "0001",
                 -1 + 3j: "0110", -1 + 1j: "0111", -1 - 3j: "0100", -1 - 1j: "0101",
                  3 + 3j: "1010",  3 + 1j: "1011",  3 - 3j: "1000",  3 - 1j: "1001",
                  1 + 3j: "1110",  1 + 1j: "1111",  1 - 3j: "1100",  1 - 1j: "1101"}
      levels = np.array([-3, -1, 1, 3])

      ref_arr = np.array([levels[i] + 1j * levels[k] for i in range(len(levels)) for k in range(len(levels))])
      #for i in range(len(levels)):
        #for k in range(len(levels)):
          #ref_arr.append(levels[i] + 1j * levels[k])
      #ref_arr = np.array(ref_arr)
      #norms = np.zeros(len(ref_arr))
      out_symbol = np.array([ref_arr[np.argmin([np.linalg.norm(ref_arr[sec_ind] - self.mapped[index]) for sec_ind in range(len(ref_arr))])] for index in range(len(self.mapped))])
      #for index in range(len(self.mapped)):
        #for sec_ind in range(len(ref_arr)):
          #norms[sec_ind] = np.linalg.norm(ref_arr[sec_ind] - self.mapped[index])
        #out_symbol.append(ref_arr[np.argmin(norms)])
      #out_symbol = np.array(out_symbol)
      self.out_bits = [dict_16[out_symbol[index]] for index in range(len(self.mapped))]
      #for index in range(len(self.mapped)):
        #self.out_bits.append(dict_16[out_symbol[index]])
      self.out_bits = np.array([int(digit) for string in self.out_bits for digit in string])

    elif self.mod_type == "QPSK":
      dict_qpsk = {1 + 1j: "11", -1 +1j: "01", -1 -1j: "00", 1 - 1j: "10"}
      levels = np.array([-1, 1])
      ref_arr = np.array([levels[i] + 1j * levels[k] for i in range(len(levels)) for k in range(len(levels))])
      #for i in range(len(levels)):
        #for k in range(len(levels)):
          #ref_arr.append(levels[i] + 1j * levels[k])
      #ref_arr = np.array(ref_arr)
      #norms = np.zeros(len(ref_arr))
      out_symbol = np.array([ref_arr[np.argmin([np.linalg.norm(ref_arr[sec_ind] - self.mapped[index]) for sec_ind in range(len(ref_arr))])] for index in range(len(self.mapped))])
      #for index in range(len(self.mapped)):
       # for sec_ind in range(len(ref_arr)):
        #  norms[sec_ind] = np.linalg.norm(ref_arr[sec_ind] - self.mapped[index])
        #out_symbol.append(ref_arr[np.argmin(norms)])
      #out_symbol = np.array(out_symbol)
      self.out_bits = [dict_16[out_symbol[index]] for index in range(len(self.mapped))]
      #for index in range(len(self.mapped)):
        #self.out_bits.append(dict_qpsk[out_symbol[index]])
      self.out_bits = np.array([int(digit) for string in self.out_bits for digit in string])

    return self.out_bits

In [3]:
def ber_count(start_bits, proc_bits):
    ber = np.sum(proc_bits != start_bits) / len(start_bits)
    return ber

def Noise_generator(mapped, snr):
    mapped_number = len(mapped)
    root_points_power = np.linalg.norm(mapped) / np.sqrt(mapped_number)
    noise_power = root_points_power / (2 * (10 ** (snr / 10)))
    Re = np.random.normal(scale = np.sqrt(noise_power), size = mapped_number)
    Im = np.random.normal(scale = np.sqrt(noise_power), size = mapped_number)
    return Re + 1j * Im

In [42]:
num_bits = 12
### test 16QAM with noise
obj_16 = MyModulator(num_bits, "16QAM", 3)
print(obj_16.bits_to_symbols(), obj_16.bits_to_symbols().shape, "\n")
print(obj_16.mapper(), "obj_16 mapper\n")
w_noise_16 = obj_16.mapper() + Noise_generator(obj_16.mapped, obj_16.snr)
print(w_noise_16, "obj_16 mapper with noise\n")
#obj.plot_constellation()
### test QPSK
#qpsk = MyModulator(num_bits, snr = 3)
#print(qpsk.bits_to_symbols(), qpsk.bits_to_symbols().shape, "\n")
#print(qpsk.mapper(), "qpsk mapper\n")
#w_noise_qpsk = qpsk.mapper() + Noise_generator(qpsk.mapped, qpsk.snr)
#print(w_noise_qpsk, "qpsk mapper with noise\n")
### test 64QAM
#obj_64 = MyModulator(num_bits, "64QAM", 3)
#print(obj_64.bits_to_symbols(), obj_64.bits_to_symbols().shape, "\n")
#print(obj_64.mapper(), "obj mapper\n")
#w_noise_64 = obj_64.mapper() + Noise_generator(obj_64.mapped, obj_64.snr)
#print(w_noise_64, "obj mapper with noise\n")
### demodulator with noise
dem16 = MyDemodulator(w_noise_16, "16QAM")
print(dem16.demodulate(), ber_count(obj_16.bits_stream, dem16.out_bits))
#dem_qpsk = MyDemodulator(w_noise_qpsk)
#print(dem_qpsk.demodulate(), ber_count(qpsk.bits_stream, dem_qpsk.out_bits))

[[1 1 1 0]
 [1 0 1 0]
 [0 0 0 0]] (3, 4) 

[ 1.+3.j  3.+3.j -3.-3.j] obj_16 mapper

[ 1.02103922+1.94012226j  3.09230986+1.97456658j -1.83474083-3.33558379j] obj_16 mapper with noise

[1 1 1 1 1 0 1 1 0 1 0 0] 0.25
